# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
# prompt: library opencv

!pip install opencv-python


In [2]:
!pip install ultralytics
!pip install opencv-p

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 658.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 13.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement opencv-p (from versions: none)
ERROR: No matching distribution found for opencv-p


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [30]:
import cv2
from ultralytics import YOLO

model_path = '/content/drive/MyDrive/T5/week6/yolov8n.pt'
model = YOLO(model_path)

100%|██████████| 6.25M/6.25M [00:00<00:00, 21.3MB/s]


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [15]:
video_path = '/content/YOLO_Video.mp4'

In [16]:
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [17]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [18]:
width

1920

In [19]:
height

1080

In [28]:
fps

29

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [21]:
output_path = '/content/output_video_with_predictions.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [30]:
def yolobbox2bbox(x,y,w,h):
    x1 = int((x - w / 2) * dw)
    x2 = int((x + w / 2) * dw)
    y1 = int((y - h / 2) * dh)
    y2 = int((y + h / 2) * dh)

    if x1 < 0:
        x1 = 0
    if x2 > dw - 1:
        x2 = dw - 1
    if y1 < 0:
        y1 = 0
    if y2 > dh - 1:
        y2 = dh - 1

    return  x1, y1, x2, y2

In [31]:
# normalized xywh from pixel values
def get_yolo_format(x1, y1, x2, y2, img_width, img_height):
    x = ((x1 + x2) / 2) / img_width
    y = ((y1 + y2) / 2) / img_height
    w = (x2 - x1) / img_width
    h = (y2 - y1) / img_height
    return (x, y, w, h)

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [22]:
color = (0, 255, 0)
thickness = 3
font = cv2.FONT_HERSHEY_SIMPLEX

In [23]:
_ , frame = cap.read()

In [24]:
frame

array([[[ 41,  77,  72],
        [ 54,  90,  85],
        [ 38,  79,  73],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[ 22,  58,  53],
        [ 49,  85,  80],
        [ 33,  74,  68],
        ...,
        [ 56,  52,  48],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[  6,  42,  37],
        [ 40,  76,  71],
        [ 34,  75,  69],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 54,  50,  46]],

       ...,

       [[128, 118, 194],
        [111, 101, 177],
        [ 99,  87, 163],
        ...,
        [108, 117, 123],
        [108, 117, 123],
        [108, 117, 123]],

       [[117, 100, 183],
        [101,  84, 167],
        [112, 100, 164],
        ...,
        [110, 119, 125],
        [108, 117, 123],
        [108, 117, 123]],

       [[ 99,  82, 165],
        [106,  89, 172],
        [126, 114, 178],
        ...,
        [110, 119, 125],
        [110, 119, 125],
        [110, 119, 125]]

In [25]:
# the boundry box you want
cv2.rectangle(frame, (100, 300), (800, 600), color, thickness)

array([[[ 41,  77,  72],
        [ 54,  90,  85],
        [ 38,  79,  73],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[ 22,  58,  53],
        [ 49,  85,  80],
        [ 33,  74,  68],
        ...,
        [ 56,  52,  48],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[  6,  42,  37],
        [ 40,  76,  71],
        [ 34,  75,  69],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 54,  50,  46]],

       ...,

       [[128, 118, 194],
        [111, 101, 177],
        [ 99,  87, 163],
        ...,
        [108, 117, 123],
        [108, 117, 123],
        [108, 117, 123]],

       [[117, 100, 183],
        [101,  84, 167],
        [112, 100, 164],
        ...,
        [110, 119, 125],
        [108, 117, 123],
        [108, 117, 123]],

       [[ 99,  82, 165],
        [106,  89, 172],
        [126, 114, 178],
        ...,
        [110, 119, 125],
        [110, 119, 125],
        [110, 119, 125]]

In [29]:

# دوال لتحديد إذا كانت السيارة على الجانب الأيسر أو الأيمن
leftside = []
rightside = []

def checkside(center, id):
    if center[0] < width // 2:  # إذا كان المركز في الجانب الأيسر
        if id not in leftside:
            leftside.append(id)
        if id in rightside:
            rightside.remove(id)
    else:  # إذا كان المركز في الجانب الأيمن
        if id not in rightside:
            rightside.append(id)
        if id in leftside:
            leftside.remove(id)

ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    height, width, _ = frame.shape

    # إجراء التتبع (استبدل `model.track` بالدالة أو النموذج الخاص بك)
    results = model.track(frame, persist=True)  # استبدل model بنموذج التتبع الذي تستخدمه

    # معالجة النتائج
    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()  # إحداثيات الصندوق المحيط
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1  # معرف الفئة
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0  # درجة
            id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1

            # تحويل إحداثيات الصندوق إلى أعداد صحيحة
            x1, y1, x2, y2 = map(int, bbox)

            # رسم المربع
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            label1 = f'ID: {id} Class: {class_id} Conf: {conf:.2f}'
            x_center = (x2 + x1) // 2
            y_center = (y1 + y2) // 2
            centers = [x_center, y_center]
            checkside(centers, id)
            label2 = f'Cars on left side: {len(leftside)} Cars on right side: {len(rightside)}'

            cv2.putText(frame, label1, (x1, y1 - 10), font, 0.5, color, thickness, cv2.LINE_AA)
            cv2.putText(frame, label2, (10, height - 10), font, 0.5, color, thickness, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


## Save and Submit Your Work

Submit both the notebook and the output video